In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#data collection
dataset=pd.read_csv("Social_Network_Ads.csv")
print(dataset)

#preprocess
#drop/remove user ID, model creation
dataset=dataset.drop("User ID",axis=1)
dataset

#preprocess Gender to 0 or 1
dataset=pd.get_dummies(dataset,drop_first=True).astype(int)
print(dataset.columns)

#Balanced Dataset or Imbalanced Dataset
print(dataset["Purchased"].value_counts())

#input output split
independent=dataset[['Age', 'EstimatedSalary', 'Gender_Male']]
dependent=dataset[['Purchased']]

print(independent.shape)
print(dependent.shape)

#Train Test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent, dependent, test_size=0.30,random_state=0)

#preprocess standard Scaler
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

      User ID  Gender  Age  EstimatedSalary  Purchased
0    15624510    Male   19            19000          0
1    15810944    Male   35            20000          0
2    15668575  Female   26            43000          0
3    15603246  Female   27            57000          0
4    15804002    Male   19            76000          0
..        ...     ...  ...              ...        ...
395  15691863  Female   46            41000          1
396  15706071    Male   51            23000          1
397  15654296  Female   50            20000          1
398  15755018    Male   36            33000          0
399  15594041  Female   49            36000          1

[400 rows x 5 columns]
Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')
Purchased
0    257
1    143
Name: count, dtype: int64
(400, 3)
(400, 1)


In [11]:
#Model creation
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
from sklearn.svm import SVC

#Model creation using Grid
from sklearn.model_selection import GridSearchCV
param_grid = {'kernel':['rbf','poly','sigmoid','linear'],
              'C':[10,100,1000,2000,3000]}
classifier = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
# fitting the model for grid search 
classifier.fit(X_train, y_train) 

Fitting 5 folds for each of 20 candidates, totalling 100 fits


C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']},
             scoring='f1_weighted', verbose=3)

In [12]:
#Evaluation Matrix: Confusion_matrix
re=classifier.cv_results_
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
#Evaluation Matrix: calculate P R F
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, y_pred)
print(clf_report)

[[73  6]
 [ 4 37]]
              precision    recall  f1-score   support

           0       0.95      0.92      0.94        79
           1       0.86      0.90      0.88        41

    accuracy                           0.92       120
   macro avg       0.90      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120



In [17]:
# print best parameter after tuning 
#print(classifier.best_params_) 
re=classifier.cv_results_
from sklearn.metrics import f1_score
f1_weighted=f1_score(y_test,y_pred,average='weighted')
print("The weighted F1-score for best parameter {}:".format(classifier.best_params_),f1_weighted)

The weighted F1-score for best parameter {'C': 10, 'kernel': 'rbf'}: 0.9171245421245421


In [18]:
#Roc = ROC (Receiver Operating Characteristic) Curve
#AUC (Area Under the Curve)
from sklearn.metrics import roc_auc_score
print("ROC-AUC Score:",roc_auc_score(y_test,classifier.predict_proba(X_test)[:,1]))
table=pd.DataFrame.from_dict(re)
table

ROC-AUC Score: 0.9654214263661625


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.020704,0.003292,0.014194,0.001955,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.892857,0.875644,0.841398,0.947015,0.946153,0.900613,0.041029,1
1,0.020620,0.003371,0.014739,0.005035,10,poly,"{'C': 10, 'kernel': 'poly'}",0.833024,0.799537,0.737557,0.928571,0.888158,0.837369,0.066798,7
2,0.014851,0.001539,0.012611,0.002590,10,sigmoid,"{'C': 10, 'kernel': 'sigmoid'}",0.769053,0.733523,0.610390,0.753871,0.759910,0.725349,0.058652,20
3,0.025927,0.002375,0.012365,0.001404,10,linear,"{'C': 10, 'kernel': 'linear'}",0.835985,0.782971,0.644599,0.927778,0.890114,0.816289,0.098849,11
4,0.022817,0.002606,0.011688,0.001889,100,rbf,"{'C': 100, 'kernel': 'rbf'}",0.855314,0.892857,0.859435,0.929144,0.928571,0.893064,0.031996,2
5,0.059167,0.013526,0.009979,0.001257,100,poly,"{'C': 100, 'kernel': 'poly'}",0.833024,0.799537,0.737557,0.928571,0.907658,0.841269,0.070136,6
6,0.012828,0.001608,0.011005,0.001309,100,sigmoid,"{'C': 100, 'kernel': 'sigmoid'}",0.769053,0.714286,0.750000,0.753871,0.759910,0.749424,0.018709,16
7,0.060563,0.013933,0.009235,0.000958,100,linear,"{'C': 100, 'kernel': 'linear'}",0.835985,0.782971,0.644599,0.927778,0.890114,0.816289,0.098849,11
8,0.054613,0.009971,0.009093,0.000584,1000,rbf,"{'C': 1000, 'kernel': 'rbf'}",0.835985,0.875644,0.858503,0.911105,0.890114,0.874270,0.025784,4
9,0.383181,0.118767,0.009592,0.001107,1000,poly,"{'C': 1000, 'kernel': 'poly'}",0.833024,0.799537,0.715601,0.928571,0.907658,0.836878,0.076857,8


In [16]:
#save model
import pickle
filename="final_Sav_Model.sav"
pickle.dump(classifier,open(filename,'wb'))
load_model=pickle.load(open("final_Sav_Model.sav",'rb'))

#apply StandardScaler to user input
preinput=sc.transform([[46,41000,1]])
print(preinput)
result=load_model.predict(preinput)
result

[[ 0.79762394 -0.8330751   0.99288247]]


C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([1])